In [33]:
import json
import re

In [34]:
# Note that you need to move any external files (images usually) either to the local director of this script or two a url
# and update your Jupyter notebook with the urls.

# A bug that I have not fixed is if a ordered or unordered list terminates as the last line in a cell then the last
# list item is closed without closing the enclosed paragraph (which is really annoying). 
# The simple fix is to add an end of line in the Jupyter notebook anytime one of these environments ends a cell.

In [35]:
# Read in the notebook file

json_file = open('../STAT_411/10 - Decision Trees.ipynb')
data = json.load(json_file)

In [36]:
# cells is an array of the cell data from the notebook

cells = data['cells']

In [37]:
current = 0

In [38]:
def new_section(p, current, lev):
    out = ''
    if current == 1:
        out += '<section>\n <title>'
        out += p[lev:]
        out += '</title>'
    elif current == 2:
        out += '<subsection>\n <title>'
        out += p[lev:]
        out += '</title>'
    elif current == 3:
        out += '<subsubsection>\n <title>'
        out += p[lev:]
        out += '</title>'
    elif current == 4:
        out += '<paragraphs>\n <title>'
        out += p[lev:]
        out += '</title>'
        
    return out

In [39]:
def close_section(current):
    out = ''
    
    if current == 1:
        out += '\n</section>\n'
    elif current == 2:
        out += '\n</subsection>\n'
    elif current == 3:
        out += '\n</subsubsection>\n'
    elif current == 4:
        out += '\n</paragraphs>\n'
    
    return out

In [40]:
def remove_colon(p):
    
    if p[-2]==':':
        return p[:-2]+p[-1]
    else:
        return p
    

In [41]:
def is_quote_envr(p, envr):
    
    out = ''
    
    if p[0]=='>' and not '>' in envr:     # If this is the start of a quote block, use the first line as title
        envr += ['>']
        if ':' in p:
            p = remove_colon(p)
        out += '<blockquote><paragraphs><title>'+p[1:-1]+'</title>\n'
        p = ''
    elif p[0]=='>':   # If this is the middle of the quote block, return a new p
        p = p[1:].lstrip()
    elif envr[-1]=='>':  # If this is the line after the end of a quote block, adjust envr and close quote block
        envr = envr[:-1]
        out += '</paragraphs></blockquote>\n'
        
    return out, p, envr

In [42]:
def is_enumerate_envr(p, envr):
    
    out = ''
    
    if len(p)<2:
        pt = 'XX'
    else:
        pt = p
    
    if str.isdigit(p[0]) and p[1]=='.':   # Are we in an ordered list
        if not 'enumerate' in envr:         # check if the environment has been set to ordered list
            envr += ['p', 'enumerate'] 
            out += '<p><ol>\n'

        temp, envr = read_line_no_enumerate(p[2:], envr)        
        out += '<li>'+temp+'</li>\n'
        p = ''      # Add the item  (need to come back and add the per character scanning here)
    elif envr[-1]=='enumerate':    # If the Jupyter file is not in an ordered list but envr still is, back out 
                                   # and close.
        envr = envr[0:-2]
        out += '</ol></p>\n'
        
    return out, p, envr
        

In [43]:
def is_itemize_envr(p, envr):
    
    out = ''
    
    if len(p)<2:
        pt = 'XX'
    else:
        pt = p
    
    if pt[0:2]=='- ':   # Are we in an unordered list
        if not 'itemize' in envr:         # check if the environment has been set to unordered list
            envr += ['p', 'itemize'] 
            out += '<p><ul>\n'
            
        temp, envr = read_line_no_itemize(p[2:], envr)        
        out += '<li>'+temp+ '</li>\n'      # Add the item  (need to come back and add the per character scanning here)
        p = ''
    elif envr[-1]=='itemize':    # If the Jupyter file is not in an ordered list but envr still is, back out 
                                   # and close.
        envr = envr[0:-2]
        out += '</ul></p>\n'
        
    return out, p, envr

In [44]:
def is_paragraph(p, envr):
    
    out = ''
    
    if len(p)==0:   # Pad the string if it is less than 1 character -- note less than one character means it has 
                    # been processed. Should we instead skip this function in the call?
        pt = '\n'
    else:
        pt = p
    
    if pt[0]!='\n' and envr[-1]!='p':  # Check if we should start a new paragraph
        envr += ['p']
        out = '<p>'
    elif (p=='\n' or p=='') and envr[-1]=='p': # Check if we should close the current paragraph
        envr = envr[0:-1]
        out = '</p>'
        
    return out, p, envr

In [45]:
def render_link(p):
    
    out = ''
    
    end1 = p.index(']')
    end2 = p.index(')')
    
    text = p[1:end1]
    href = p[end1+2:end2]
    
    out = '<url href="'+href+'"> '+text+' </url> '
    p = p[end2+1:]
    
    return out, p

In [46]:
def render_em(p):
    
    # A note here:  Markdown includes * and ** for italics and then bold. We will translate these in Pretext as the <em> and
    # <alert> tags resp..
    
    out = ''
    
    end1 = p.index('*')
    if p[end1+1]=='*':
        end2 = p[(end1+2):].index('*')
        text = p[(end1+2):end2+2]
        out = '<alert>' + text + '</alert>'
        p = p[end2+4:]
    else:
        end2 = p[(end1+1):].index('*')
        text = p[end1+1:end2+1]
        out = '<em>' + text + '</em>'
        p = p[end2+2:]
        
    return out, p

In [47]:
def render_img(p):
    
    # The basic issue here is that we have to render the image code all at once as otherwise the quote environment
    # will catch it when the bracket is closed with '>'
    
    out = ''
    
    equal = p.index('=')
    end = p.index('>')
    if '"' in p[equal:end]:
        source_start = p[equal:].index('"')+equal
        source_end = p[source_start+1:].index('"')+source_start+1
    elif "'" in p[equal:end]:
        source_start = p[equal:].index("'")+equal
        source_end = p[source_start+1:].index("'")+source_start+1
    else: 
        print('No quote in image environment.')
    out = '<image source=' + p[source_start:source_end+1] + 'width="60%"/>'
    p = p[end+1:]
    
    # Note that we are defaulting to a size specification of 60% - you will need to go through and adjust that. 
    # The issue is that Jupyter is using pixel specification and Pretext wants percentage of available width.
    
    # Using Figure with empty caption?
    
    return out, p

In [48]:
def remove_br(p):
    
    # Markdown requires <br> commands for carriage returns. Pretext has other methods (at the moment I'm just using <p>)
    
    end = p.index('>')
    
    return p[end+1:]

In [49]:
def close_envr(envr):
    
    out = ''
    
    for code in envr[-1::-1]:
        if code == '':
            pass
        elif code == 'p':
            out += '</p>\n'
        elif code == 'enumerate':
            out += '</ol>\n'
        elif code == 'itemize':
            out += '</ul>\n'
        elif code == '>':
             out += '</paragraphs></blockquote>\n'            
        else:
            print('Environment Code '+code+' Not Recognized')
            
    return out

In [50]:
def render_math(p):
    
    # For now assuming mathematics is all on one line. Will need to update in the future for formulas
    # that run over multiple lines.
    
    if p[1] == '$':
        start = 2
        out = '<me>'
        close = '</me>'
    else:
        start = 1
        out = '<m>'
        close = '</m>'
    
    end = p[start:].index('$')
    
    out += p[start:end+1] + close  # A little bothered by the +1 here...
    p = p[end+2*start:]
    
    return out, p
    

In [51]:
def read_line_no_enumerate(p, envr):
    
    out = ''
    
    while p!='': # Basically go through string character by character, though some control expressions are 
                 # handled as multiple characters.
        
            # Some nesting of environments is not allowed:  bullet within bullet will not work.
        
            # Check for quote environment (assumption for now is that quote environment is 
            # not nested in another envr)
        
            temp, p, envr = is_quote_envr(p, envr)
            out += temp
            
            # Check for itemize environment, which could be nested in a quote or enumerate environment (will need
            # to come back and fix this so the nesting could go the other way).
            
            if len(p)>2:
                if p[0:1]=='- ' or envr[-1]=='itemize':
                    temp, p, envr = is_itemize_envr(p, envr)
                    out += temp
            
            # Check if we need a paragraph environment - note that enumerate (and later list come with their own)
            
            if len(p)>0 or envr[-1]=='p':
                temp, p, envr = is_paragraph(p, envr)
                out += temp
            
            # Read by characters, looking for links, math environments, and other text markdown
            
            if len(p) > 0:
                if p[0]=='[' and ']' in p:
                    temp, p = render_link(p)
                    out += temp
                elif p[0]=='*':
                    temp, p = render_em(p)
                    out += temp
                elif p[0]=='$':
                    temp, p = render_math(p)
                    out += temp
                elif p[0:4]=='<img':
                    temp, p = render_img(p)
                    out += temp
                else:
                    out += p[0]
                    p = p[1:]
                    
    return out, envr

In [52]:
def read_line_no_itemize(p, envr):
    
    out = ''
    
    while p!='': # Basically go through string character by character, though some control expressions are 
                 # handled as multiple characters.
        
            # Some nesting of environments is not allowed:  bullet within bullet will not work.
        
            # Check for quote environment (assumption for now is that quote environment is 
            # not nested in another envr)
        
            temp, p, envr = is_quote_envr(p, envr)
            out += temp
        
            # Assumption for now is that enumerate is not nested in any environment other than quote - will
            # need to adjust this later.
            
            if len(p)>2:        
                if (str.isdigit(p[0]) and p[1]=='.') or envr[-1]=='enumerate':
                    temp, p, envr = is_enumerate_envr(p, envr)
                    out += temp
            
            # Check if we need a paragraph environment - note that enumerate (and later list come with their own)
            
            if len(p)>0 or envr[-1]=='p':
                temp, p, envr = is_paragraph(p, envr)
                out += temp
            
            # Read by characters, looking for links, math environments, and other text markdown
            
            if len(p) > 0:
                if p[0]=='[' and ']' in p:
                    temp, p = render_link(p)
                    out += temp
                elif p[0]=='*':
                    temp, p = render_em(p)
                    out += temp
                elif p[0]=='$':
                    temp, p = render_math(p)
                    out += temp
                elif p[0:4]=='<img':
                    temp, p = render_img(p)
                    out += temp
                else:
                    out += p[0]
                    p = p[1:]
                    
    return out, envr

In [53]:
def read_line(p, envr):
    
    out = ''
    
    while p!='': # Basically go through string character by character, though some control expressions are 
                 # handled as multiple characters.
        
            # Some nesting of environments is not allowed:  bullet within bullet will not work.
        
            # Check for quote environment (assumption for now is that quote environment is 
            # not nested in another envr)
        
            temp, p, envr = is_quote_envr(p, envr)
            out += temp
        
            # Check for enumerate environment, which could be nested in a quote environment (will need to come 
            # back and fix this so the nesting could go the other way).
        
            # Assumption for now is that enumerate is not nested in any environment other than quote - will
            # need to adjust this later.
      
            if len(p)>2 or envr[-1]=='enumerate':
                temp, p, envr = is_enumerate_envr(p, envr)
                out += temp
            
            # Check for itemize environment, which could be nested in a quote or enumerate environment (will need
            # to come back and fix this so the nesting could go the other way).
            
            if len(p)>2 or envr[-1]=='itemize':
                temp, p, envr = is_itemize_envr(p, envr)
                out += temp
            
            # Check if we need a paragraph environment - note that enumerate (and later list come with their own)
            
            if len(p)>0 or 'p' in envr:
                temp, p, envr = is_paragraph(p, envr)
                out += temp
            
            # Read by characters, looking for links, math environments, and other text markdown
            
            if len(p) > 0:
                if p[0]=='[' and ']' in p:
                    temp, p = render_link(p)
                    out += temp
                elif p[0]=='*':
                    temp, p = render_em(p)
                    out += temp
                elif p[0]=='$':
                    temp, p = render_math(p)
                    out += temp
                elif p[0:4]=='<img':
                    temp, p = render_img(p)
                    out += temp
                else:
                    out += p[0]
                    p = p[1:]
                    
    return out, envr

In [54]:
def manipulate_sections(p, current):
        
        out = ''
        
        lev = p.count('#')  # Check if this line is identifying a section and then compare with current section
                            # position and adjust.
        
        if not lev == 0:
            while current > lev:
                out += close_section(current)
                current += -1
            if current == lev:
                out += close_section(current)
                out += new_section(p, current, lev)
                p = '\n'
            while current < lev-1:
                current += 1
                out += new_section('\n', current, lev)
            if current == lev-1:
                current += 1
                out += new_section(p, current, lev)
                p = '\n'
        
        return out, p, current

In [55]:
def read_metadata(cell, current):
    out = ''
    temp = ''

    envr = ['']  # Initialize an list to keep track of open environments assumption will be that all 
                   # multiline environments are identified in markdown by the first character of the line.
                   # quote environment is terminated when the > symbol is dropped.
                   # enumerate and bullet environments either when the enclosing environment increments or 
                    # at the termination of the block. 
    
    for p in cell:  # cycle through each line of this cell
        if p[0]=='#': # Check if we are changing sectioning
            temp, p, current = manipulate_sections(p, current)
            out += temp
    
        temp, envr = read_line(p, envr)
        out += temp
    
    # at the end of a markdown block close any envr left open
        
    out += close_envr(envr)
    
    return out, current 

In [56]:
def read_code(source, outputs=None):
    
#    out = '<sage language="python">\n'

    # 7-bit C1 ANSI sequences
    ansi_escape = re.compile(r'''
    \x1B    # ESC
    [@-_]   # 7-bit C1 Fe
    [0-?]*  # Parameter bytes
    [ -/]*  # Intermediate bytes
    [@-~]   # Final byte
    ''', re.VERBOSE)

    out = '<console>\n'
    
    out += '<input>\n'+''.join(source).replace('<', '&lt;')+'\n</input>\n'
    
    if outputs:
        
        # outputs can be a string on its own, or a list of strings for multiline display
        
        temp = ''
        
        if type(outputs)==list:
            for p in outputs:
                strip = ansi_escape.sub('', p)  # ocassionally Jupyter has formated the output to include ASCII control
                                              # Characters. This snippet removes them and gives a clean string.
                temp += strip
        else:
            temp = ansi_escape.sub('', outputs) # ocassionally Jupyter has formated the output to include ASCII control
                                                # Characters. This snippet removes them and gives a clearn string.
            
        out += '<output>\n'+''.join(temp).replace('<', '&lt;')+'\n</output>\n'
        
#    out += '</sage>\n'
    
    out += '</console>\n'
    
    return out    

In [57]:
pretext_head = '''<pretext>

    <docinfo>
        <macros>
        \\newcommand{\doubler}[1]{2#1}
        </macros>
    </docinfo>

    <article xml:id="minimal">
        <title>Conversion Script Test</title>

        <frontmatter>

            <titlepage>
                <author>
                    <personname>Virgil U Pierce</personname>
                    <institution>University of Northern Colorado</institution>
                </author>
                <date><today /></date>
            </titlepage>

            <abstract>
                <p>Testing of conversion of a Jupyter notebook to Pretext, and compiling</p>
            </abstract>
            
            <introduction>

                <p>Check if process.call worked</p>

                <p>This is a short paragraph to introduce the article (but it is not the abstract).  
                It is optional, in case it would be preferable to have the first section be titled an 
                <q>Introduction.</q></p>
            </introduction>
            
        </frontmatter>'''

In [58]:
pretext_tail = '''</article>
</pretext>'''

In [59]:
def add_block(cell, current):
    
    out = ''
    
    if cell['cell_type']=='markdown':
        out, current = read_metadata(cell['source'], current)
    elif cell['cell_type']=='code':
        out += '<listing>\n'
        if cell['outputs']:
            if 'data' in cell['outputs'][0]:
                out += read_code(cell['source'], cell['outputs'][0]['data']['text/plain'])
            elif 'text' in cell['outputs'][0]:
                out += read_code(cell['source'], cell['outputs'][0]['text'])
        else:
            out += read_code(cell['source'])
        out += '</listing>\n'
    else:
        print('Unkown Cell Type Found')
        
    return out, current

In [60]:
import sys

sys.executable

'/anaconda3/bin/python'

In [61]:
current = 0

out = ''

for cell in cells:
    print('Reading Cell {}'.format(cells.index(cell)) )
    temp, current = add_block(cell, current)
    out += temp
        
# Final step for the block metadata read, close any currently open environments that have not already
# been closed.
    
# However List, Enumerate, and Quote environments will be assumed to terminate within a block.
    
for i in range(current, 0, -1):   
    out += close_section(i)

Reading Cell 0
Reading Cell 1
Reading Cell 2
Reading Cell 3
Reading Cell 4
Reading Cell 5
Reading Cell 6
Reading Cell 7
Reading Cell 8
Reading Cell 9
Reading Cell 10
Reading Cell 11
Reading Cell 12
Reading Cell 13
Reading Cell 14
Reading Cell 15
Reading Cell 16
Reading Cell 17
Reading Cell 18
Reading Cell 19
Reading Cell 20
Reading Cell 21
Reading Cell 22
Reading Cell 23
Reading Cell 24
Reading Cell 25
Reading Cell 26
Reading Cell 27
Reading Cell 28
Reading Cell 29
Reading Cell 30
Reading Cell 31
Reading Cell 32
Reading Cell 33
Reading Cell 34
Reading Cell 35
Reading Cell 36
Reading Cell 37
Reading Cell 38
Reading Cell 39
Reading Cell 40
Reading Cell 41
Reading Cell 42
Reading Cell 43
Reading Cell 44
Reading Cell 45
Reading Cell 46
Reading Cell 47
Reading Cell 48
Reading Cell 49
Reading Cell 50
Reading Cell 51
Reading Cell 52
Reading Cell 53
Reading Cell 54
Reading Cell 55
Reading Cell 56
Reading Cell 57
Reading Cell 58
Reading Cell 59
Reading Cell 60
Reading Cell 61
Reading Cell 62
Re

In [62]:
pretext = pretext_head + out + pretext_tail

In [63]:
text_file = open('test.xml', "w")

text_file.write(pretext)

text_file.close()

In [64]:
%%bash
xsltproc ../mathbook/xsl/mathbook-html.xsl test.xml

test.xml:745: parser error : EntityRef: expecting ';'
   RoofMatl_WdShngl RoofMatl_Roll RoofMatl_Tar&Grv RoofMatl_WdShake  \
                                                  ^
test.xml:778: parser error : EntityRef: expecting ';'
       'RoofMatl_Roll', 'RoofMatl_Tar&Grv', 'RoofMatl_WdShake',
                                         ^
test.xml:846: parser error : EntityRef: expecting ';'
   RoofMatl_WdShngl  RoofMatl_Roll  RoofMatl_Tar&Grv  RoofMatl_WdShake  \
                                                    ^
unable to parse test.xml


CalledProcessError: Command 'b'xsltproc ../mathbook/xsl/mathbook-html.xsl test.xml\n'' returned non-zero exit status 6.